# Przetwarzanie Danych - Przygotowanie do Modelowania

Ten notebook zawiera wszystkie kroki przetwarzania danych niezbędne do trenowania modeli prognozowania popytu.

## Cele przetwarzania:
- Czyszczenie i filtrowanie danych
- Tworzenie cech czasowych
- Agregacja danych do poziomia dziennego/tygodniowego
- Przygotowanie sekwencji dla modeli LSTM
- Podział na zbiory treningowe/walidacyjne/testowe

In [ ]:
# Import bibliotek
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
import joblib
import sys
sys.path.append('../src')

## 1. Ładowanie surowych danych

In [ ]:
# Ładowanie danych z poprzedniego etapu

## 2. Czyszczenie danych

In [ ]:
# Usuwanie duplikatów, obsługa braków danych, filtrowanie outlierów

## 3. Tworzenie cech czasowych

In [ ]:
# Rok, miesiąc, dzień tygodnia, kwartał, sezonowość

## 4. Agregacja danych

In [ ]:
# Agregacja do poziomu dziennego - łączna sprzedaż, liczba transakcji

## 5. Tworzenie cech dodatkowych

In [ ]:
# Średnie ruchome, wskaźniki lag, cechy statystyczne

## 6. Skalowanie danych

In [ ]:
# Normalizacja/standaryzacja zmiennych numerycznych

## 7. Przygotowanie sekwencji dla LSTM

In [ ]:
# Tworzenie sekwencji wejściowych i etykiet dla modeli rekurencyjnych

## 8. Podział na zbiory treningowe/walidacyjne/testowe

In [ ]:
# Chronologiczny podział danych z zachowaniem kolejności czasowej

## 9. Zapisanie przetworzonych danych

In [ ]:
# Zapis do plików pickle/csv dla dalszego wykorzystania